
# HDA Flowsheet Simulation and Optimization


## Learning outcomes


- Construct a steady-state flowsheet using the IDAES unit model library
- Connecting unit models in a  flowsheet using Arcs
- Using the SequentialDecomposition tool to initialize a flowsheet with recycle
- Fomulate and solve an optimization problem
    - Defining an objective function
    - Setting variable bounds
    - Adding additional constraints 


## Problem Statement

Hydrodealkylation is a chemical reaction that often involves reacting
an aromatic hydrocarbon in the presence of hydrogen gas to form a
simpler aromatic hydrocarbon devoid of functional groups. In this
example, toluene will be reacted with hydrogen gas at high temperatures
 to form benzene via the following reaction:

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> → C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>**


This reaction is often accompanied by an equilibrium side reaction
which forms diphenyl, which we will neglect for this example.

This example is based on the 1967 AIChE Student Contest problem as
present by Douglas, J.M., Chemical  Design of Chemical Processes, 1988,
McGraw-Hill.

The flowsheet that we will be using for this module is shown below with the stream conditions. We will be processing toluene and hydrogen to produce at least 370 TPY of benzene. As shown in the flowsheet, we use a flash tank, F101, to separate out the non-condensibles, and a distillation column, D101, to further separate the benzene-toluene mixture to improve the benzene purity.  The non-condensibles separated out in F101 will be partially recycled back to M101 and the rest will be purged. We will assume ideal gas for this flowsheet. The properties required for this module are defined in

- `hda_ideal_VLE.py`
- `hda_reaction_kinetic.py`

The state variables chosen for the property package are **flows of component by phase, temperature and pressure**. The components considered are: **toluene, hydrogen, benzene and methane**. Therefore, every stream has 8 flow variables, 1 temperature and 1 pressure variable. 

![](HDA_flowsheet_distillation.png)




## Importing required pyomo and idaes components


To construct a flowsheet, we will need several components from the pyomo and idaes package. Let us first import the following components from Pyomo:
- Constraint (to write constraints)
- Var (to declare variables)
- ConcreteModel (to create the concrete model object)
- Expression (to evaluate values as a function of variables defined in the model)
- Objective (to define an objective function for optimization)
- SolverFactory (to solve the problem)
- TransformationFactory (to apply certain transformations)
- Arc (to connect two unit models)
- SequentialDecomposition (to initialize the flowsheet in a sequential mode)

For further details on these components, please refer to the pyomo documentation: https://pyomo.readthedocs.io/en/latest/


In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

Import `Arc` and `SequentialDecomposition` tools from `pyomo.network`

In [2]:
# Import the above mentioned tools from pyomo
from pyomo.network import Arc, SequentialDecomposition

From idaes, we will be needing the FlowsheetBlock and the following unit models:
- Mixer
- Heater
- StoichiometricReactor
- Flash
- Separator (splitter) 
- PressureChanger

In [3]:
from idaes.core import FlowsheetBlock

In [4]:
from idaes.generic_models.unit_models import (PressureChanger,
                                              Mixer,
                                              Separator as Splitter,
                                              Heater,
                                              CSTR,
                                              Flash,
                                              Translator)

from idaes.generic_models.unit_models.distillation import TrayColumn
from idaes.generic_models.unit_models.distillation.condenser \
    import CondenserType, TemperatureSpec 

We will also be needing some utility tools to put together the flowsheet and calculate the degrees of freedom. 

In [5]:
# Utility tools to put together the flowsheet and calculate the degrees of freedom
from idaes.generic_models.unit_models.pressure_changer import ThermodynamicAssumption
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state

# Import idaes logger to set output levels
import idaes.logger as idaeslog

## Importing required thermo and reaction packages

Finally, we import the thermo (`hda_ideal_VLE.py`) and reaction package (`hda_reaction_kineric.py`) for the HDA process. We have created a custom thermo package that assumes Ideal Gas with support for VLE. The reaction package consists of the stochiometric coefficients for the reaction, heat of reaction, and kinetic information (Arrhenius constant and activation energy). 

We use another thermo package, `BTXParameterBlock`, which does not contain non-condensables (i.e., methane and hydrogen) to simplify the VLE calculations in the `TrayColumn` block.

In [6]:
import hda_reaction_kinetic as reaction_props
from hda_ideal_VLE import HDAParameterBlock
from idaes.generic_models.properties.activity_coeff_models.\
    BTX_activity_coeff_VLE import BTXParameterBlock

## Constructing the Flowsheet

We have now imported all the components, unit models, and property modules we need to construct a flowsheet. Let us create a ConcreteModel and add the flowsheet block as we did in the earlier module. 

In [7]:
# Create a Pyomo Concrete Model to contain the problem
m = ConcreteModel()

# Add a steady state flowsheet block to the model
m.fs = FlowsheetBlock(default={"dynamic": False})

We now need to add the property packages to the flowsheet. Unlike Module 1, where we only had a thermo property package, for this flowsheet we will also need to add a reaction property package. 

In [8]:
# Property package for benzene, toluene, hydrogen, methane mixture
m.fs.thermo_params = HDAParameterBlock()

# Property package for the benzene-toluene mixture
m.fs.bt_properties = BTXParameterBlock(default={
        "valid_phase": ('Liq', 'Vap'),
        "activity_coeff_model": "Ideal"
})

# Reaction package for the HDA reaction
m.fs.reaction_params = reaction_props.HDAReactionParameterBlock(
        default={"property_package": m.fs.thermo_params})

## Adding Unit Models

Let us start adding the unit models we have imported to the flowsheet. Here, we are adding the Mixer (assigned a name M101) and a Heater (assigned a name H101). Note that, all unit models need to be given a property package argument. In addition to that, there are several arguments depending on the unit model, please refer to the documentation for more details (https://idaes-pse.readthedocs.io/en/latest/model_libraries/core_lib/unit_models/index.html). For example, the Mixer unit model here is given a `list` consisting of names to the three inlets. 

In [9]:
# Adding the mixer M101 to the flowsheet
m.fs.M101 = Mixer(default={"property_package": m.fs.thermo_params,
                           "inlet_list": ["toluene_feed", "hydrogen_feed", "vapor_recycle"]})

# Adding the heater H101 to the flowsheet
m.fs.H101 = Heater(default={"property_package": m.fs.thermo_params,
                            "has_pressure_change": False,
                            "has_phase_equilibrium": True})

Let us now add the CSTR (assign the name R101) and pass the following arguments:
- "property_package": m.fs.thermo_params
- "reaction_package": m.fs.reaction_params
- "has_heat_of_reaction": True 
- "has_heat_transfer": True
- "has_pressure_change": False

In [10]:
# Add reactor with the specifications above
m.fs.R101 = CSTR(
            default={"property_package": m.fs.thermo_params,
                     "reaction_package": m.fs.reaction_params,
                     "has_heat_of_reaction": True,
                     "has_heat_transfer": True,
                     "has_pressure_change": False})

Let us now add the Flash (assign the name F101), Splitter (assign the name S101) and PressureChanger (assign the name C101)

In [11]:
# Adding the flash tank F101 to the flowsheet
m.fs.F101 = Flash(default={"property_package": m.fs.thermo_params,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
                           
# Adding the splitter S101 to the flowsheet
m.fs.S101 = Splitter(default={"property_package": m.fs.thermo_params,
                              "ideal_separation": False,
                              "outlet_list": ["purge", "recycle"]})

# Adding the compressor C101 to the flowsheet    
m.fs.C101 = PressureChanger(default={
            "property_package": m.fs.thermo_params,
            "compressor": True,
            "thermodynamic_assumption": ThermodynamicAssumption.isothermal})                             

## Remark

Currently, the `SequentialDecomposition()` tool which we will later be using to initialize the flowsheet does not support distillation column. So, we will first simulate the flowsheet without the distillation column. After it converges, we will then add the distillation column, initialize it, and then simulate the entire flowsheet.

## Translator block

Benzene and toluene are separated by distillation, so the process involves phase equilibrium and two-phase flow conditions. However, the presence of hydrogen and methane complicates the calculations. This is because, hydrogen and methane are non-condensable under all conditions of interest; ergo, a vapor phase will always be present, and the mixture bubble point is extremely low. To simplify the phase equilibrium calculations, hydrogen and methane will be considered completely as non-condensable and insoluble in the liquid outlet from the flash separator F101.

Since no hydrogen and methane will be present in the unit operations following the flash, a different component list can be used to simplify the property calculations. IDAES supports this by allowing the definition of multiple property packages within a single flowsheet and the use of `Translator` blocks to convert between different property calculations, component lists, and equations of state. 

For this flowsheet, we use the `m.fs.thermo_params` package, which contains all the four species, for the reactor loop, and the simpler `m.fs.bt_properties` for unit operations following the flash. We define a `Translator` block to link the source property package and the package it is to be translated to in the following manner:

In [12]:
# Add translator block to convert between property packages       
m.fs.translator = Translator(default={
        "inlet_property_package": m.fs.thermo_params,
        "outlet_property_package": m.fs.bt_properties
}) 

### Translator block constraints

The `Translator` block needs to know how to translate between the two property packages. This must be custom coded for each application because of the generality of the IDAES framework.

For this process, five constraints are required based on the state variables used in the outgoing process.

- Since we assumed that only benzene and toluene are present in the liquid phase, the total molar flowrate must be the sum of molar flowrates of benzene and toluene, respectively.
- Temperature of the inlet and outlet streams must be the same.
- Pressure of the inlet and outgoing streams must be the same
- The mole fraction of benzene in the outgoing stream is the ratio of the molar flowrate of liquid benzene in the inlet to the sum of molar flowrates of liquid benzene and toluene in the inlet.
- The mole fraction of toluene in the outgoing stream is the ratio of the molar flowrate of liquid toluene in the inlet to the sum of molar flowrates of liquid benzene and toluene in the inlet.

In [13]:
# Add constraint: Total flow = benzene flow + toluene flow (molar)
m.fs.translator.eq_total_flow = Constraint(
    expr=m.fs.translator.outlet.flow_mol[0] ==
    m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"] +
    m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"])

# Add constraint: Outlet temperature = Inlet temperature
m.fs.translator.eq_temperature = Constraint(
    expr=m.fs.translator.outlet.temperature[0] ==
    m.fs.translator.inlet.temperature[0])

# Add constraint: Outlet pressure = Inlet pressure
m.fs.translator.eq_pressure = Constraint(
    expr=m.fs.translator.outlet.pressure[0] ==
    m.fs.translator.inlet.pressure[0])

# Add constraint: Benzene mole fraction definition
m.fs.translator.eq_mole_frac_benzene = Constraint(
    expr=m.fs.translator.outlet.mole_frac_comp[0, "benzene"] ==
    m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"] /
    (m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"] +
    m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"]))

# Add constraint: Toluene mole fraction definition
m.fs.translator.eq_mole_frac_toluene = Constraint(
    expr=m.fs.translator.outlet.mole_frac_comp[0, "toluene"] ==
    m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"] /
    (m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"] +
    m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"]))

Finally, let's add the Heater H102 in the same way as H101 but pass the m.fs.bt_properties thermodynamic package. We will add the distillation column after converging the flowsheet.

In [14]:
# Add the Heater H102 to the flowsheet
m.fs.H102 = Heater(default={"property_package": m.fs.bt_properties,
                            "has_pressure_change": True,
                            "has_phase_equilibrium": True})   

## Connecting Unit Models using Arcs

We have now added the initial set of unit models to the flowsheet. However, we have not yet specifed how the units are connected. To do this, we will be using the `Arc` which is a pyomo component that takes in two arguments: `source` and `destination`. Let us connect the outlet of the mixer (M101) to the inlet of the heater (H101). 

In [15]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)


![](HDA_flowsheet_distillation.png) 

We will now be connecting the rest of the units as shown below. Notice how the outlet names are different for the flash tank as it has a vapor and a liquid outlet. 

In [16]:
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)
m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.F101.inlet)
m.fs.s06 = Arc(source=m.fs.F101.vap_outlet, destination=m.fs.S101.inlet)
m.fs.s08 = Arc(source=m.fs.S101.recycle, destination=m.fs.C101.inlet)
m.fs.s09 = Arc(source=m.fs.C101.outlet,
                   destination=m.fs.M101.vapor_recycle)
m.fs.s10a = Arc(source=m.fs.F101.liq_outlet,
                   destination=m.fs.translator.inlet)
m.fs.s10b = Arc(source=m.fs.translator.outlet,
                   destination=m.fs.H102.inlet)                   

We have now connected the unit model block using the arcs. However, each of these arcs link to ports on the two unit models that are connected. In this case, the ports consist of the state variables that need to be linked between the unit models. Pyomo provides a convenient method to write these equality constraints for us between two ports and this is done as follows:

In [17]:
TransformationFactory("network.expand_arcs").apply_to(m)

### Appending additional constraints to the model

Now, we will see how we can add additional constraints to the model using `Constraint` from Pyomo.

Consider the reactor R101. By default, the conversion of a component is not calculated when we simulate the flowsheet. If we are interested either in specifying or constraining the conversion value, we can add the following constraint to calculate the conversion:
$$ \text{Conversion of toluene} = \frac{\text{molar flow of toluene in the inlet} - \text{molar flow of toluene in the outlet}}{\text{molar flow of toluene in the inlet}} $$ 

We add the constraint to the model as shown below.

In [18]:
# Define the conversion variables using 'Var'
m.fs.R101.conversion = Var(initialize=0.75, bounds=(0, 1))

# Append the constraint to the model
m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion*m.fs.R101.inlet.
    flow_mol_phase_comp[0, "Vap", "toluene"] ==
    (m.fs.R101.inlet.flow_mol_phase_comp[0, "Vap", "toluene"] -
    m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "toluene"]))

In the above, note that the variable flow_mol_phase_comp has the index - [time, phase, component]. As this is a steady-state flowsheet, the time index by default is 0. The valid phases are ["Liq", "Vap"]. Similarly the valid component list is ["benzene", "toluene", "hydrogen", "methane"].

## Fixing feed conditions and Initializing the flowsheet

Let us first check how many degrees of freedom exist for this flowsheet using the `degrees_of_freedom` tool we imported earlier. 

In [19]:
print(degrees_of_freedom(m))

29


In [20]:
# Check the degrees of freedom
assert degrees_of_freedom(m) == 29

We will now be fixing the toluene feed stream to the conditions shown in the flowsheet above. Please note that though this is a pure toluene feed, the remaining components are still assigned a very small non-zero value to help with convergence and initializing. 

In [21]:
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(0.30)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.temperature.fix(303.2)
m.fs.M101.toluene_feed.pressure.fix(350000)

Similarly, let us fix the hydrogen feed to the following conditions in the next cell:
      <ul>
         <li>F<sub>H2</sub> = 0.30 mol/s</li>
         <li>F<sub>CH4</sub> = 0.02 mol/s</li>
         <li>Remaining components = 1e-5 mol/s</li>
         <li>T = 303.2 K</li>
         <li>P = 350000 Pa</li>
      </ul>

In [22]:
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(0.30)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(0.02)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.hydrogen_feed.temperature.fix(303.2)
m.fs.M101.hydrogen_feed.pressure.fix(350000)

### Fixing unit model specifications

Now that we have fixed our inlet feed conditions, we will now be fixing the operating conditions for the unit models in the flowsheet. Let us set set the H101 outlet temperature to 600 K. 

In [23]:
# Fix the temperature of the outlet from the heater H101
m.fs.H101.outlet.temperature.fix(600)

Set the conditions for the reactor R101 to the following conditions:
- `conversion` = 0.75 
- `heat_duty` = 0

In [24]:
# Fix the 'conversion' of the reactor R101
m.fs.R101.conversion.fix(0.75)

# Fix the 'heat_duty' of the reactor R101
m.fs.R101.heat_duty.fix(0)

The Flash conditions for F101 can be set as follows. 

In [25]:
# Fix the temperature of the vapor outlet from F101
m.fs.F101.vap_outlet.temperature.fix(325.0)

# Fix the pressure drop in the flash F101
m.fs.F101.deltaP.fix(0)

Let us fix the split fraction of the purge stream from the splitter S101 and the outlet pressure from the compressor C101

In [26]:
# Fix the split fraction of the 'purge' stream from S101
m.fs.S101.split_fraction[0, "purge"].fix(0.2)

# Fix the pressure of the outlet from the compressor C101
m.fs.C101.outlet.pressure.fix(350000)

Finally, let us fix the temperature of the outlet from H102 and the pressure drop in H102 as the following

In [27]:
# Fix the temperature of the outlet from the heater H102
m.fs.H102.outlet.temperature.fix(375)

# Fix the pressure drop in the heater H102
m.fs.H102.deltaP.fix(-200000)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now defined all the feed conditions and the inputs required for the unit models. The system should now have 0 degrees of freedom i.e. should be a square problem. Please check that the degrees of freedom is 0. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [28]:
# Check the degrees of freedom
print(degrees_of_freedom(m))

0


In [29]:
# Check the degrees of freedom
assert degrees_of_freedom(m) == 0

### Initialization

This subsection will demonstrate how to use the built-in sequential decomposition tool to initialize our flowsheet.

Let us first create an object for the `SequentialDecomposition` and specify our options for this. 

In [30]:
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 5

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

Which is the tear stream? Display tear set and order

In [31]:
for o in heuristic_tear_set:
    print(o.name)

fs.s03


What sequence did the SD tool determine to solve this flowsheet with the least number of tears? 

In [32]:
for o in order:
    print(o[0].name)

fs.H101
fs.R101
fs.F101
fs.S101
fs.C101
fs.M101


The SequentialDecomposition tool has determined that the tear stream is the mixer outlet (s03 in the Figure above). We will need to provide a reasonable guess for this.

In [33]:
tear_guesses = {
        "flow_mol_phase_comp": {
                (0, "Vap", "benzene"): 1e-5,
                (0, "Vap", "toluene"): 1e-5,
                (0, "Vap", "hydrogen"): 0.30,
                (0, "Vap", "methane"): 0.02,
                (0, "Liq", "benzene"): 1e-5,
                (0, "Liq", "toluene"): 0.30,
                (0, "Liq", "hydrogen"): 1e-5,
                (0, "Liq", "methane"): 1e-5},
        "temperature": {0: 303},
        "pressure": {0: 350000}}

# Pass the tear_guess to the SD tool
seq.set_guesses_for(m.fs.H101.inlet, tear_guesses)

Next, we need to tell the tool how to initialize a particular unit. We will be writing a python function which takes in a "unit" and calls the initialize method on that unit.

In [34]:
def function(unit):
        unit.initialize(outlvl=idaeslog.INFO)

We are now ready to initialize our flowsheet in a sequential mode. Note that we specifically set the iteration limit to be 5 as we are trying to use this tool only to get a good set of initial values such that IPOPT can then take over and solve this flowsheet for us. 

In [35]:
seq.run(m, function)

2021-05-28 09:35:40 [INFO] idaes.init.fs.H101.control_volume: Initialization Complete
2021-05-28 09:35:41 [INFO] idaes.init.fs.H101: Initialization Complete: optimal - Optimal Solution Found
2021-05-28 09:35:41 [INFO] idaes.init.fs.R101.control_volume: Initialization Complete
2021-05-28 09:35:41 [INFO] idaes.init.fs.R101: Initialization Complete: optimal - Optimal Solution Found
2021-05-28 09:35:41 [INFO] idaes.init.fs.F101.control_volume: Initialization Complete
2021-05-28 09:35:41 [INFO] idaes.init.fs.F101: Initialization Complete: optimal - Optimal Solution Found
2021-05-28 09:35:41 [INFO] idaes.init.fs.S101.purge_state: Initialization Complete
2021-05-28 09:35:41 [INFO] idaes.init.fs.S101.recycle_state: Initialization Complete
2021-05-28 09:35:41 [INFO] idaes.init.fs.S101: Initialization Step 2 Complete: optimal - Optimal Solution Found
2021-05-28 09:35:41 [INFO] idaes.init.fs.translator.properties_out: Initialization Step 1 optimal - Optimal Solution Found.
2021-05-28 09:35:41 [IN

We have now initialized the flowsheet. Let us run the flowsheet in a simulation mode to look at the results.

In [36]:
# Create the solver object
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6, 'max_iter': 5000}

# Solve the model
results = solver.solve(m, tee=True)

Ipopt 3.13.2: tol=1e-06
max_iter=5000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        

### Add distillation column 

This completes the simulation of the flowsheet constructed earlier. We will now 
- Add the distillation column 
- Connect it to the heater 
- Add the necessary equality constraints
- Propogate the state variable information from the outlet of the heater to the inlet of the distillation column 
- Fix the degrees of freedom of the distillation block (reflux ratio, boilup ratio, and condenser pressure)
- Initialize the distillation block.



In [37]:
# Add distillation column to the flowsheet
m.fs.D101 = TrayColumn(default={
                        "number_of_trays": 10,
                        "feed_tray_location": 5,
                        "condenser_type":
                            CondenserType.totalCondenser,
                        "condenser_temperature_spec":
                            TemperatureSpec.atBubblePoint,
                        "property_package": m.fs.bt_properties,
                        "has_heat_transfer": False,
                        "has_pressure_change": False})

# Connect the outlet from the heater H102 to the distillation column
m.fs.s11 = Arc(source=m.fs.H102.outlet, 
               destination=m.fs.D101.feed)

# Add the necessary equality constraints
TransformationFactory("network.expand_arcs").apply_to(m)

# Propagate the state
propagate_state(m.fs.s11)

# Fix the reflux ratio, boilup ratio, and the condenser pressure
m.fs.D101.condenser.reflux_ratio.fix(0.5)
m.fs.D101.reboiler.boilup_ratio.fix(0.5)
m.fs.D101.condenser.condenser_pressure.fix(150000)

# Initialize the distillation column
m.fs.D101.initialize(outlvl=idaeslog.INFO)                        

perties_in_liq: Initialization Step 5 optimal - Optimal Solution Found.
2021-05-28 09:36:01 [INFO] idaes.init.fs.D101.rectification_section[4].properties_in_vap: Initialization Step 1 optimal - Optimal Solution Found.
2021-05-28 09:36:01 [INFO] idaes.init.fs.D101.rectification_section[4].properties_in_vap: Initialization Step 2 optimal - Optimal Solution Found.
2021-05-28 09:36:01 [INFO] idaes.init.fs.D101.rectification_section[4].properties_in_vap: Initialization Step 3 optimal - Optimal Solution Found.
2021-05-28 09:36:01 [INFO] idaes.init.fs.D101.rectification_section[4].properties_in_vap: Initialization Step 4 optimal - Optimal Solution Found.
2021-05-28 09:36:01 [INFO] idaes.init.fs.D101.rectification_section[4].properties_in_vap: Initialization Step 5 optimal - Optimal Solution Found.
2021-05-28 09:36:01 [INFO] idaes.init.fs.D101.rectification_section[4].properties_out: Initialization Step 1 optimal - Optimal Solution Found.
2021-05-28 09:36:01 [INFO] idaes.init.fs.D101.rectifica

## Adding expressions to compute capital and operating costs

In this section, we will add a few Expressions that allow us to evaluate the performance. Expressions provide a convenient way of calculating certain values that are a function of the variables defined in the model. For more details on Expressions, please refer to: https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Expressions.html

In [38]:
# Expression to compute the total cooling cost
m.fs.cooling_cost = Expression(expr=0.25e-7 * (-m.fs.F101.heat_duty[0]) +
                                0.2e-7 * (-m.fs.D101.condenser.heat_duty[0]))

# Expression to compute the total heating cost
m.fs.heating_cost = Expression(expr=2.2e-7 * m.fs.H101.heat_duty[0] +
                                1.2e-7 * m.fs.H102.heat_duty[0] +
                                1.9e-7 * m.fs.D101.reboiler.heat_duty[0])

# Expression to compute the total operating cost
m.fs.operating_cost = Expression(expr=(3600 * 24 * 365 *
                                (m.fs.heating_cost + m.fs.cooling_cost)))

# Expression to compute the total capital cost
m.fs.capital_cost = Expression(expr=1e5*m.fs.R101.volume[0])    

### Solve the entire flowsheet

In [39]:
# Check that the degrees of freedom is zero
assert degrees_of_freedom(m) == 0

In [40]:
solver.solve(m, tee=True)

Ipopt 3.13.2: tol=1e-06
max_iter=5000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1169, 'Number of variables': 1169, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.20345544815063477}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

## Analyze the Results of the Square Problem

How much is the total cost (operating_cost + capital_cost), operating_cost, capital_cost, benzene purity in the distillate from the distilation column, and conversion of toluene in the reactor?

In [41]:
print('total cost = $', value(m.fs.capital_cost) + value(m.fs.operating_cost))
print('operating cost = $', value(m.fs.operating_cost))
print('capital cost = $', value(m.fs.capital_cost))
print()
print('Distillate flowrate = ', value(m.fs.D101.condenser.distillate.flow_mol[0]()), 'mol/s')
print('Benzene purity = ', 100 * value(m.fs.D101.\
    condenser.distillate.mole_frac_comp[0, "benzene"]), '%')
print('Residue flowrate = ', value(m.fs.D101.reboiler.bottoms.flow_mol[0]()), 'mol/s')
print('Toluene purity = ', 100 * value(m.fs.D101.\
    reboiler.bottoms.mole_frac_comp[0, "toluene"]), '%')
print()
print('Conversion = ', 100 * value(m.fs.R101.conversion), '%')
print()
print('Overhead benzene loss in F101 = ', \
    100 * value(m.fs.F101.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"]) / \
        value(m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "benzene"]), '%' )

total cost = $ 442297.7468818147
operating cost = $ 427593.00669734733
capital cost = $ 14704.74018446735

Distillate flowrate =  0.16202588062179787 mol/s
Benzene purity =  89.51393273342501 %
Residue flowrate =  0.10509317979151479 mol/s
Toluene purity =  43.323757274199885 %

Conversion =  75.0 %

Overhead benzene loss in F101 =  42.161938483603244 %


In [42]:
# Check solver solve status
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

import pytest
assert value(m.fs.operating_cost) == pytest.approx(427593.007, abs=100)
assert value(m.fs.capital_cost) == pytest.approx(14704.740, abs=100)

Get the state of the streams entering and leaving the reactor R101

In [43]:
m.fs.R101.report()


Unit : fs.R101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key       : Value   : Fixed : Bounds
    Heat Duty :  0.0000 :  True : (None, None)
       Volume : 0.14705 : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                               Inlet     Outlet  
    flow_mol_phase_comp ('Liq', 'benzene')  1.2993e-07 1.2993e-07
    flow_mol_phase_comp ('Liq', 'toluene')  8.4147e-07 8.4147e-07
    flow_mol_phase_comp ('Liq', 'methane')  1.0000e-08 1.0000e-08
    flow_mol_phase_comp ('Liq', 'hydrogen') 1.0000e-08 1.0000e-08
    flow_mol_phase_comp ('Vap', 'benzene')     0.11936    0.35374
    flow_mol_phase_comp ('Vap', 'toluene')     0.31252   0.078129
    flow_mol_phase_comp ('Vap', 'methane')      1.0377     1.2721
    flow_mol_phase_comp 

Get the state of the streams entering and leaving the reactor R101

In [44]:
m.fs.F101.report()


Unit : fs.F101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value   : Fixed : Bounds
          Heat Duty : -70343. : False : (None, None)
    Pressure Change :  0.0000 :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                               Inlet    Vapor Outlet  Liquid Outlet
    flow_mol_phase_comp ('Liq', 'benzene')  1.2993e-07   1.0000e-08       0.20460  
    flow_mol_phase_comp ('Liq', 'toluene')  8.4147e-07   1.0000e-08      0.062520  
    flow_mol_phase_comp ('Liq', 'methane')  1.0000e-08   1.0000e-08    2.6712e-07  
    flow_mol_phase_comp ('Liq', 'hydrogen') 1.0000e-08   1.0000e-08    2.6712e-07  
    flow_mol_phase_comp ('Vap', 'benzene')     0.35374      0.14915    1.0000e-08  
    flow_mol_phase_comp ('Vap'

Next, let's look at how much benzene we are loosing with the light gases out of F101. IDAES has tools for creating stream tables based on the `Arcs` and/or `Ports` in a flowsheet. Let us create and print a simple stream table showing the stream leaving the reactor and the vapor stream from F101.

How much benzene are we loosing in the F101 vapor outlet stream?

In [45]:
from idaes.core.util.tables import create_stream_table_dataframe, stream_table_dataframe_to_string

st = create_stream_table_dataframe({"Reactor": m.fs.s05, "Light Gases": m.fs.s06})
print(stream_table_dataframe_to_string(st))

                                          Reactor   Light Gases
flow_mol_phase_comp ('Liq', 'benzene')  1.2993e-07  1.0000e-08 
flow_mol_phase_comp ('Liq', 'toluene')  8.4147e-07  1.0000e-08 
flow_mol_phase_comp ('Liq', 'methane')  1.0000e-08  1.0000e-08 
flow_mol_phase_comp ('Liq', 'hydrogen') 1.0000e-08  1.0000e-08 
flow_mol_phase_comp ('Vap', 'benzene')     0.35374     0.14915 
flow_mol_phase_comp ('Vap', 'toluene')    0.078129    0.015610 
flow_mol_phase_comp ('Vap', 'methane')      1.2721      1.2721 
flow_mol_phase_comp ('Vap', 'hydrogen')    0.32821     0.32821 
temperature                                 771.85      325.00 
pressure                                3.5000e+05  3.5000e+05 


You can querry additional variables here if you like. 

# Optimization


We saw from the results above that the total operating cost for the base case was $442,297 per year. We are producing 0.162 mol/s of benzene at a purity of 89.5%. However, we are losing around 43.3% of benzene in F101 vapor outlet stream. 

Let us try to minimize this cost such that:
- we are producing at least 0.18 mol/s of benzene as distillate i.e. our product stream
- purity of benzene i.e. the mole fraction of benzene in the distillate is at least 99%
- restricting the benzene loss in F101 vapor outlet to less than 20%

For this problem, our decision variables are as follows:
- H101 outlet temperature
- R101 outlet temperature
- F101 outlet temperature
- H102 outlet temperature
- Condenser pressure
- reflux ratio
- boilup ratio


Let us declare our objective function for this problem. 

In [46]:
m.fs.objective = Objective(expr=m.fs.operating_cost + m.fs.capital_cost)

Now, we need to unfix the decision variables as we had solved a square problem (degrees of freedom = 0) until now. 

In [47]:
m.fs.H101.outlet.temperature.unfix()
m.fs.R101.conversion.unfix()  
m.fs.F101.vap_outlet.temperature.unfix()
m.fs.D101.condenser.condenser_pressure.unfix()
m.fs.D101.condenser.reflux_ratio.unfix()
m.fs.D101.reboiler.boilup_ratio.unfix()
m.fs.H102.outlet.temperature.unfix()

Next, we need to set bounds on these decision variables to values shown below:

 - H101 outlet temperature [500, 600] K
 - R101 outlet temperature [600, 900] K
 - F101 outlet temperature [298, 450] K
 - H102 outlet temperature [350, 400] K
 - D101 condenser pressure [101325, 150000] Pa
 - D101 reflux ratio [0.1, 5]
 - D101 boilup ratio [0.1, 5]

In [48]:
# Set bounds on the temperature of the outlet from H101
m.fs.H101.outlet.temperature[0].setlb(500)
m.fs.H101.outlet.temperature[0].setub(600)

# Set bounds on the temperature of the outlet from R101
m.fs.R101.outlet.temperature[0].setlb(600)
m.fs.R101.outlet.temperature[0].setub(900)

# Set bounds on the volume of the reactor R101
m.fs.R101.volume[0].setlb(0)

# Set bounds on the temperature of the vapor outlet from F101
m.fs.F101.vap_outlet.temperature[0].setlb(298)
m.fs.F101.vap_outlet.temperature[0].setub(450.0)

# Set bounds on the temperature of the outlet from H102
m.fs.H102.outlet.temperature[0].setlb(350)
m.fs.H102.outlet.temperature[0].setub(400)

# Set bounds on the pressure inside the condenser 
m.fs.D101.condenser.condenser_pressure.setlb(101325)
m.fs.D101.condenser.condenser_pressure.setub(150000)

# Set bounds on the reflux ratio
m.fs.D101.condenser.reflux_ratio.setlb(0.1)
m.fs.D101.condenser.reflux_ratio.setub(5)

# Set bounds on the boilup ratio
m.fs.D101.reboiler.boilup_ratio.setlb(0.1)
m.fs.D101.reboiler.boilup_ratio.setub(5)

Now, the only things left to define are our constraints on overhead loss in F101, distillate flowrate and its purity. Let us first look at defining a constraint for the overhead loss in F101 where we are restricting the benzene leaving the vapor stream to less than 20 % of the benzene available in the reactor outlet. 

In [49]:
# Ensure that the overhead loss of benzene from F101 <= 20% 
m.fs.overhead_loss = Constraint(
    expr=m.fs.F101.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"] <=
    0.20 * m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "benzene"])

Now, add the constraint such that we are producing at least 0.18 mol/s of benzene in the product stream which is the distillate of D101. Let us name this constraint as m.fs.product_flow. 

In [50]:
# Add minimum product flow constraint
m.fs.product_flow = Constraint(
    expr=m.fs.D101.condenser.distillate.flow_mol[0] >= 0.18)

Let us add the final constraint on product purity or the mole fraction of benzene in the distillate such that it is at least greater than 99%. 

In [51]:
m.fs.product_purity = Constraint(
    expr=m.fs.D101.condenser.
    distillate.mole_frac_comp[0, "benzene"] >= 0.99)


We have now defined the optimization problem and we are now ready to solve this problem. 




In [52]:
results = solver.solve(m, tee=True)

Ipopt 3.13.2: tol=1e-06
max_iter=5000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        

## Optimization Results

Display the results and product specifications

In [53]:
print('total cost = $', value(m.fs.capital_cost) + value(m.fs.operating_cost))
print('operating cost = $', value(m.fs.operating_cost))
print('capital cost = $', value(m.fs.capital_cost))
print()
print('Distillate flowrate = ', value(m.fs.D101.condenser.distillate.flow_mol[0]()), 'mol/s')
print('Benzene purity = ', 100 * value(m.fs.D101.\
    condenser.distillate.mole_frac_comp[0, "benzene"]), '%')
print('Residue flowrate = ', value(m.fs.D101.reboiler.bottoms.flow_mol[0]()), 'mol/s')
print('Toluene purity = ', 100 * value(m.fs.D101.\
    reboiler.bottoms.mole_frac_comp[0, "toluene"]), '%')
print()
print('Conversion = ', 100 * value(m.fs.R101.conversion), '%')
print()
print('Overhead benzene loss in F101 = ', \
    100 * value(m.fs.F101.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"]) / \
        value(m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "benzene"]), '%' )

total cost = $ 438269.11213246576
operating cost = $ 408342.3527037184
capital cost = $ 29926.75942874737

Distillate flowrate =  0.17999999002647055 mol/s
Benzene purity =  98.99999900051326 %
Residue flowrate =  0.1084428638896574 mol/s
Toluene purity =  15.859460669687527 %

Conversion =  93.28983757156041 %

Overhead benzene loss in F101 =  17.444853021293365 %


Display optimal values for the decision variables

In [54]:
print('Optimal Values')
print()

print('H101 outlet temperature = ', value(m.fs.H101.outlet.temperature[0]), 'K')

print()
print('R101 outlet temperature = ', value(m.fs.R101.outlet.temperature[0]), 'K')

print()
print('F101 outlet temperature = ', value(m.fs.F101.vap_outlet.temperature[0]), 'K')

Optimal Values

H101 outlet temperature =  568.7845207063697 K

R101 outlet temperature =  790.0196711917349 K

F101 outlet temperature =  298.1499971259889 K


In [55]:
# Check solver solve status
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

import pytest
assert value(m.fs.operating_cost) == pytest.approx(408342.352, abs=100)
assert value(m.fs.capital_cost) == pytest.approx(29926.759, abs=100)